# Demo IoT Sensor Data Input
**Simulation the sensor sensing environment and sending data into our system, which setup in [1.Setup_Env](./1.Setup_Env.ipynb), Step 1~5.**

## 1. Import Required Lib

In [49]:
import json
import csv
import requests
import time
import os
import copy

## 2. Setup Env Var
**\_\_AIOTCORE\_\_ is the aiot-core publice interface's IP address of aiot-core container.**

**\_\_IOTA\_UL\_\_  and \_\_IOTA\_JSON\_\_ are the iot-agent publice interface's IP address of both container.**

In [50]:
__AIOTCORE__="http://127.0.0.1:9250"
__IOTA_UL__="http://127.0.0.1:7896"
__IOTA_JSON__="http://127.0.0.1:7897"

## 3. Show All Usuable Dataset Name

In [51]:
dirlist=os.listdir("../../Dataset/NAB/")
try:
    dirlist.remove("README.md")
except:
    pass
try:
    dirlist.remove("combined_windows.json")
except:
    pass
filelist=[]
for dirname in dirlist:
    try:
        file=os.listdir(f"../../Dataset/NAB/{dirname}/")
        for i in file:
            filelist.append(f"{dirname}/{i}")
    except:
        pass
filedict={x:[] for x in filelist}
print("available dataset name:")
for file_name in filedict:
    print(file_name)

available dataset name:
artificialWithAnomaly/art_daily_jumpsdown.csv
artificialWithAnomaly/art_daily_flatmiddle.csv
artificialWithAnomaly/art_daily_nojump.csv
artificialWithAnomaly/art_load_balancer_spikes.csv
artificialWithAnomaly/art_daily_jumpsup.csv
artificialWithAnomaly/art_increase_spike_density.csv
realTraffic/occupancy_t4013.csv
realTraffic/speed_7578.csv
realTraffic/speed_6005.csv
realTraffic/TravelTime_387.csv
realTraffic/speed_t4013.csv
realTraffic/TravelTime_451.csv
realTraffic/occupancy_6005.csv
realKnownCause/ambient_temperature_system_failure.csv
realKnownCause/machine_temperature_system_failure.csv
realKnownCause/rogue_agent_key_updown.csv
realKnownCause/ec2_request_latency_system_failure.csv
realKnownCause/rogue_agent_key_hold.csv
realKnownCause/cpu_utilization_asg_misconfiguration.csv
realKnownCause/nyc_taxi.csv
realAdExchange/exchange-3_cpc_results.csv
realAdExchange/exchange-2_cpc_results.csv
realAdExchange/exchange-3_cpm_results.csv
realAdExchange/exchange-2_cpm_r

## 4. Pick Wanted Dataset and Load Data Into Memory

In [52]:
wanted=["artificialWithAnomaly/art_daily_jumpsdown.csv",
        "artificialWithAnomaly/art_daily_jumpsup.csv",
        "realKnownCause/ec2_request_latency_system_failure.csv",
        "realKnownCause/machine_temperature_system_failure.csv"]

for dirname in dirlist:
    filelist=os.listdir(f"../../Dataset/NAB/{dirname}/")
    for file in filelist:
        if f"{dirname}/{file}" not in wanted:
            continue
        with open(f"../../Dataset/NAB/{dirname}/{file}","r") as f:
            data=f.readlines()
            data.pop(0)
            for idx,line in enumerate(data):
                time=line.split(",")[0].replace(" ","T")[0:-3]
                value=float(line.split(",")[1])
                filedict[f"{dirname}/{file}"].append((time,value))
print("Data Loaded:")
for file_name,datalist in filedict.items():
    if file_name not in wanted:
        continue
    print(f"{file_name}:")
    for d in datalist[0:10]:
        print(d)
    print("...")
    

Data Loaded:
artificialWithAnomaly/art_daily_jumpsdown.csv:
('2014-04-01T00:00', 18.090486228499998)
('2014-04-01T00:05', 20.359842585899997)
('2014-04-01T00:10', 21.105469847200002)
('2014-04-01T00:15', 21.1515852522)
('2014-04-01T00:20', 18.1371405424)
('2014-04-01T00:25', 21.6256605306)
('2014-04-01T00:30', 21.6738066877)
('2014-04-01T00:35', 20.2455614926)
('2014-04-01T00:40', 19.9283327955)
('2014-04-01T00:45', 18.1794698523)
...
artificialWithAnomaly/art_daily_jumpsup.csv:
('2014-04-01T00:00', 19.761251902999998)
('2014-04-01T00:05', 20.500833287)
('2014-04-01T00:10', 19.9616414445)
('2014-04-01T00:15', 21.4902660734)
('2014-04-01T00:20', 20.1877394098)
('2014-04-01T00:25', 19.9231256718)
('2014-04-01T00:30', 21.698403961700002)
('2014-04-01T00:35', 20.8787583842)
('2014-04-01T00:40', 18.4461996294)
('2014-04-01T00:45', 18.7108178448)
...
realKnownCause/machine_temperature_system_failure.csv:
('2013-12-02T21:15', 73.96732207)
('2013-12-02T21:20', 74.93588199999998)
('2013-12-02T2

## Get Iot Agent Setting From AIOT-CORE

In [53]:
import pprint
deviceInfo={}
service_group=requests.get(f"{__AIOTCORE__}/service-groups").json()['value']
for SG in service_group:
    devices=requests.get(f"{__AIOTCORE__}/service-groups/{SG}").json()["Device"]
    for DeviceName in devices:
        url=f"{__AIOTCORE__}/devices/{SG}/{DeviceName}"
        data=requests.get(url).json()
        agentURL=data["iotagent_config"]["Iot-Agent-Url"].split(":")[0:2]+[str(data["iotagent_config"]["Device-Port"])]
        agentURL=":".join(agentURL)
        if "7897" in agentURL:
            agentURL=__IOTA_JSON__
        else:
            agentURL=__IOTA_UL__
        apikey=data["iotagent_config"]["apikey"]
        Resource=data["iotagent_config"]["Resource"]
        DeviceID=f"{SG}:{DeviceName}"
        filename=data["static_attributes"]["fieldName"]+".csv"
        for i in filedict:
            if filename in i:
                filename=i
                break
        deviceInfo[filename]={"agentURL":agentURL,"apikey":apikey,"Resource":Resource,"DeviceID":DeviceID}
pprint.pprint(deviceInfo)
filedata=copy.deepcopy(filedict)

{'artificialWithAnomaly/art_daily_jumpsdown.csv': {'DeviceID': 'SG_Artifical:Sensor01',
                                                   'Resource': '/iot/d',
                                                   'agentURL': 'http://140.116.72.250:7896',
                                                   'apikey': 'vpCNt19G3gNJPfE21V7w6oUcYY'},
 'artificialWithAnomaly/art_daily_jumpsup.csv': {'DeviceID': 'SG_Artifical:Sensor02',
                                                 'Resource': '/iot/json',
                                                 'agentURL': 'http://140.116.72.250:7897',
                                                 'apikey': 'wiUIlRLOAuS0Nv3mCxrhJnAsVP'},
 'realKnownCause/ec2_request_latency_system_failure.csv': {'DeviceID': 'SG_Real:Sensor01',
                                                           'Resource': '/iot/d',
                                                           'agentURL': 'http://140.116.72.250:7896',
                                        

## Start Feeding Data
**Delay Period is the IoT Device sensing cycle time in second, must not be smaller than 4**

In [ ]:
import time 
delay_period=4


count=0
while count!=len(wanted)-1:
    count=0
    for file,content in filedata.items():
        try:
            data=content.pop(0)
        except:
            count+=1
            continue
        device=deviceInfo[file]
        agentURL=device["agentURL"]
        apikey=device["apikey"]
        Resource=device["Resource"]
        DeviceID=device["DeviceID"]
        if "7897" in agentURL:
            header = {"Content-Type": "application/json"}
            body = {"COUNT" : str(data[1]) , "TS": data[0]}
            print(f"Request URL:{agentURL}{Resource}?k={apikey}&i={DeviceID}")
            print(f"Request Data:{body}")
            r = requests.post(agentURL+Resource+"?k="+apikey+"&i="+DeviceID,
                headers=header,
                data=json.dumps(body),
            )
            print(r.status_code,r.text)
        else:
            header = {"Content-Type": "text/plain"}
            body = "COUNT|" + str(data[1]) + "|TS|" + data[0]
            print(f"Request URL:{agentURL}{Resource}?k={apikey}&i={DeviceID}")
            print(f"Request Data:{body}")
            r = requests.post(agentURL+Resource+"?k="+apikey+"&i="+DeviceID,
                headers=header,
                data=body
            )
            print(r.status_code,r.text)
    time.sleep(int(delay_period))

print("End")